In [1]:
from scipy.integrate import odeint
from scipy.spatial.transform import Rotation
import pandas as pd
import numpy as np
import plotly.graph_objects as go

#### Creating Initial Conditions
$$
\vec{R}_{PER} = \frac{a(1-e^2)}{1+e cos(f)}
\left[
cos(f) \hat{\textbf{e}}+
sin(f) \hat{\textbf{p}}
\right]
$$

$$
\vec{V}_{PER} = \sqrt{\frac{\mu}{a(1-e^2)}}
\left[
-sin(f) \hat{\textbf{e}}+
(1+cos(f)) \hat{\textbf{p}}
\right]
$$

$$
T_{PER \rightarrow ECI} = \left[\begin{array}{c|c|c}
\cos \Omega \cos \omega - \sin \Omega \sin \omega \cos i & -\cos \Omega \sin \omega - \sin \Omega \cos \omega \cos i & \sin \Omega \sin i \\
\hline
\sin \Omega \cos \omega + \cos \Omega \sin \omega \cos i & -\sin \Omega \sin \omega + \cos \Omega \cos \omega \cos i & -\cos \Omega \sin i \\
\hline
\sin \omega \sin i & \cos \omega \sin i & \cos i
\end{array}\right]
$$

In [2]:
mu=398600.4418
wE = np.array([0,0,7.2921159e-5])

def T_PER_to_ECI(raan,i,aop):
    cos_raan = np.cos(np.radians(raan))
    sin_raan = np.sin(np.radians(raan))
    cos_i = np.cos(np.radians(i))
    sin_i = np.sin(np.radians(i))
    cos_aop = np.cos(np.radians(aop))
    sin_aop = np.sin(np.radians(aop))

    return np.array([[cos_raan * cos_aop - sin_raan * sin_aop * cos_i,-cos_raan * sin_aop - sin_raan * cos_aop * cos_i,sin_raan * sin_i],
                     [sin_raan * cos_aop + cos_raan * sin_aop * cos_i,-sin_raan * sin_aop + cos_raan * cos_aop * cos_i,-cos_raan * sin_i],
                     [sin_aop * sin_i,cos_aop * sin_i,cos_i]])

def T_ECI_to_ECEF(t):
    gamma = wE[2]*np.array(t)
    sin_gamma = np.sin(gamma)
    cos_gamma = np.cos(gamma)
    if np.shape(t):
        z = np.zeros_like(sin_gamma)
        o = np.ones_like(sin_gamma)
        return np.array([[cos_gamma,sin_gamma,z],
                         [-sin_gamma,cos_gamma,z],
                         [z,z,o]]).T
    else:
        return np.array([[cos_gamma,sin_gamma,0],
                         [-sin_gamma,cos_gamma,0],
                         [0,0,1]])

def Translation_EOM(x,t):
    r_3 = np.linalg.norm(x[:3])**3
    return np.array([x[3],x[4],x[5],-mu*x[0]/r_3,-mu*x[1]/r_3,-mu*x[2]/r_3])

In [3]:
a = 6378+700
e = 0.0
i = 80
raan = 50
aop = 20
f = 0

T = 2*np.pi*(a**3/mu)**0.5
ra = a*(1+e)

rPER_0 = a*(1-e**2)/(1+e*np.cos(np.radians(f)))*np.array([np.cos(np.radians(f)),np.sin(np.radians(f)),0])
vPER_0 = (mu/(a*(1-e**2)))**0.5*np.array([-np.sin(np.radians(f)),(e+np.cos(np.radians(f))),0])

R = T_PER_to_ECI(raan,i,aop)

rECI_0 = R@rPER_0
vECI_0 = R@vPER_0

times = np.arange(0,T,0.1)
tol = 1e-12
xECI_0 = np.hstack([rECI_0,vECI_0])
xECI = odeint(Translation_EOM,xECI_0,times,rtol=tol,atol=tol)

rECI = xECI[:,:3]
vECI = xECI[:,3:]

#### Computing ECEF State
$$
\omega_{E} = 7.2921159e^{-5} [rad/s]
$$

$$
\gamma = \omega_{E} t
$$

$$
T_{ECI\rightarrow ECEF} = \left[\begin{array}{c|c|c}
cos(\gamma)&-sin(\gamma)&0\\
\hline
sin(\gamma)&cos(\gamma)&0\\
\hline
0&0&1
\end{array}\right]
$$

$$
\vec{R}_{ECEF} = T_{ECI\rightarrow ECEF} \vec{R}_{ECI}
$$

$$
\vec{V}_{ECEF} = T_{ECI\rightarrow ECEF} \vec{V}_{ECI} + \vec{R}_{ECI}\times \vec{\omega}_E
$$

In [4]:
# Computing ECEF State

R = T_ECI_to_ECEF(times)

rECEF = np.array([r@T for r,T in zip(rECI,R)])
vECEF = np.array([v@T+np.cross(r,wE) for r,v,T in zip(rECI,vECI,R)])

In [5]:
print(np.linalg.norm(vECI[0]))
print(np.linalg.norm(vECEF[0]))

7.5043591156129805
7.430101402915084


#### Computing Gravity Gradient Torque
$$
\vec{L_g} = \frac{3\mu}{R_c^5}(\vec{R_c}\times I_c\vec{R_c})
$$

In [6]:
# Computing Gravity Gradient Torque

I = np.array([[1,0,0],
              [0,2,0],
              [0,0,3]])

Lg = np.array([3*mu/np.linalg.norm(r)**5*np.cross(r,I@r) for r in rECI])